In [106]:
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from yahoo_fin import options as oi
from pandas import ExcelWriter
from scipy.stats import norm
import yfinance as yf
import pandas as pd
import numpy as np

import datetime
import time
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
yf.pdr_override()

In [110]:
tickers = "AMZN"

In [3]:
# Assume No Dividends between now and option expiry date

# df = pdr.get_data_yahoo(tickers_1, start_date, end_date)
# dividends = yf.Ticker(tickers_1).dividends
# dividends

[*********************100%***********************]  1 of 1 completed


Date
1987-05-11    0.000536
1987-08-10    0.000536
1987-11-17    0.000714
1988-02-12    0.000714
1988-05-16    0.000714
                ...   
2020-11-06    0.205000
2021-02-05    0.205000
2021-05-07    0.220000
2021-08-06    0.220000
2021-11-05    0.220000
Name: Dividends, Length: 73, dtype: float64

In [111]:
# Risk Free Rate, r
r = 0.014

# 1. Stock Data

# 1a. Current Stock Price, S
S = si.get_live_price(ticker)

# 1b. Volatility Rate, sigma
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()
stock_data = si.get_data(ticker, start_date, end_date)
stock_data['Percent Change'] = 1 + stock_data['adjclose'].pct_change() 
stock_data['r_i'] = np.log(stock_data['Percent Change'])

r_hat = np.mean(stock_data['r_i'])
stock_data['(r_i - r_hat)^2'] = np.power((stock_data['r_i'] - r_hat), 2)

sigma = np.sum(stock_data['(r_i - r_hat)^2']) / (len(stock_data) - 2)

# 2. Option Data
options_data = oi.get_calls(ticker)
options_data = options_data[options_data['Strike'].between(0.8*S, 1.2*S)]

# 2a. Strike Price, K
K = options_data['Strike']

# 2b. Time until Expiration, T
exp_year = int(options_data['Contract Name'].iloc[0][len(ticker) : len(ticker) + 2]) + 2000
exp_month = int(options_data['Contract Name'].iloc[0][len(ticker) + 2: len(ticker) + 4])
exp_day = int(options_data['Contract Name'].iloc[0][len(ticker) + 4: len(ticker) + 6])

buy_date = datetime.datetime.now()
exp_date = datetime.datetime(exp_year, exp_month, exp_day, 16, 0, 0)
T = exp_date - buy_date
T = (T.days + T.seconds/(60*60*24))/365.25

In [112]:
# Calculate Call Price

d1 = (np.log(S / K) + ((r + (np.power(sigma, 2)/2))*T))/(sigma*np.power(T,0.5))
d2 = d1 - sigma*np.power(T, 0.5)

C = S*norm.cdf(d1) - K*np.exp(-r * T)*norm.cdf(d2)
options_data['BS_Price'] = C

In [89]:
options_data

0.0002542442191197333